<a href="https://colab.research.google.com/github/mgamzec/GeospatialData-Python/blob/main/WorkingFolium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Geospatial_Python/Udemy_geospatialCourse/Copy of MumbaiGeoMerge.csv')

In [ ]:
df.head()

,Unnamed: 0,Area,Location,Latitude,Longitude,Zone,Neighbourhood,Localty,Min,Max,Avg,Change,latitude,longitude
0,41,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,Andheri,Western Suburbs,Jogeshwari West,6328,35000,31625.39,34.5,19.136394,72.837382
1,69,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,Andheri,Western Suburbs,Andheri West,1750,118750,61324.84,43.0,19.117249,72.833968
2,126,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,Andheri,Western Suburbs,Andheri West,1750,118750,61324.84,43.0,19.117249,72.833968
3,212,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,Andheri,Western Suburbs,Jogeshwari West,6328,35000,31625.39,34.5,19.136394,72.837382
4,514,Bhayandar,"Mira-Bhayandar,Western Suburbs",19.290000,72.850000,Mira-Bhayandar,Western Suburbs,Mira Road East,678,197266,92444.61,7.2,19.281697,72.856068


In [ ]:
df.drop(['Latitude','Longitude','Location'], axis='columns', inplace=True)

In [ ]:
df.head()

,Unnamed: 0,Area,Zone,Neighbourhood,Localty,Min,Max,Avg,Change,latitude,longitude
0,41,Amboli,Andheri,Western Suburbs,Jogeshwari West,6328,35000,31625.39,34.5,19.136394,72.837382
1,69,D.N. Nagar,Andheri,Western Suburbs,Andheri West,1750,118750,61324.84,43.0,19.117249,72.833968
2,126,Four Bungalows,Andheri,Western Suburbs,Andheri West,1750,118750,61324.84,43.0,19.117249,72.833968
3,212,Lokhandwala,Andheri,Western Suburbs,Jogeshwari West,6328,35000,31625.39,34.5,19.136394,72.837382
4,514,Bhayandar,Mira-Bhayandar,Western Suburbs,Mira Road East,678,197266,92444.61,7.2,19.281697,72.856068


In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

In [ ]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
## dataframer--> nearby_venues
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Latitude',
                  'Longitude',
                  'Venue',
                  'Venue_Lat',
                  'Venue_Long',
                  'Venue_Category']

    return(nearby_venues)

In [ ]:
Mumbai_Venues = getVenues(names=df['Localty'],
                        latitudes=df['latitude'],
                        longitudes=df['longitude'])

Jogeshwari West
Andheri West
Andheri West
Jogeshwari West
Mira Road East
Bandra West
Bandra West
Dahisar
Goregaon West
Kandivali West
Kandivali West
Kandivali East
Bandra West
Bandra East
Goregaon East
Malad East
Malad West
Andheri West
Powai
Powai


In [ ]:
Mumbai_Venues.head()

,Neighbourhood,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
0,Jogeshwari West,19.136394,72.837382,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant
1,Jogeshwari West,19.136394,72.837382,Doolally Taproom,19.135917,72.833094,Brewery
2,Jogeshwari West,19.136394,72.837382,The Little Door,19.139265,72.833180,Pub
3,Jogeshwari West,19.136394,72.837382,Pizza Express,19.131893,72.834668,Pizza Place
4,Jogeshwari West,19.136394,72.837382,Mainland China,19.140391,72.838033,Chinese Restaurant


In [ ]:
Mumbai_Venues_unique_count = Mumbai_Venues['Venue_Category'].value_counts().to_frame(name='Count')

In [ ]:
Mumbai_Venues_unique_count

,Count
Indian Restaurant,217
Coffee Shop,96
Café,89
Ice Cream Shop,75
Hotel,73
...,...
Recreation Center,1
Hookah Bar,1
Sports Bar,1
Bistro,1
